In [555]:
from html import unescape
from bs4 import BeautifulSoup
import re
import random
from fake_useragent import UserAgent
import requests
from pprint import pprint
import nltk
import string
from nltk.tokenize import word_tokenize
from collections import Counter
from tqdm.auto import tqdm
from time import sleep
import time

Качаю линки на фильмы с кинопоиска

In [817]:
session = requests.session()
ua = UserAgent(verify_ssl=False)

In [818]:
url = 'https://www.kinopoisk.ru/lists/top500/?page=9&tab=all'
req = session.get(url, headers={'User-Agent': ua.chrome})
page = req.text
soup = BeautifulSoup(page, 'html.parser')

In [819]:
film_links = soup.find_all('a', {'class': 'selection-film-item-meta__link'})
films_ids = []
for link in film_links:
    link = re.findall(r'href="(.+?)">', str(link))
    films_ids.append(link[0])

In [820]:
films_ids

['/film/258328/',
 '/film/5012/',
 '/film/7226/',
 '/film/8227/',
 '/film/368/',
 '/film/42571/',
 '/film/43553/',
 '/film/102124/',
 '/film/1902/',
 '/film/7660/',
 '/film/46512/',
 '/film/279828/',
 '/film/45463/',
 '/film/42667/',
 '/film/1091/',
 '/film/39259/',
 '/film/273209/',
 '/film/363/',
 '/film/16117/',
 '/film/538/',
 '/film/3442/',
 '/film/47814/',
 '/film/63929/',
 '/film/7765/',
 '/film/505851/',
 '/film/337/',
 '/film/276376/',
 '/film/468/',
 '/film/42088/',
 '/film/43332/']

Качаю по 30 пол и отр отзывов (а также тестовые), кладу их в два списка. Также чищю текст. Дебильный кинопоиск блочит мои запросы, а ещё может не быть какого-то комментария (зачастую отрицательного). Насколько я понимаю, это почти нереально пофиксить. Поэтому недостающее я добавил ручками в конце (то есть брал брал другую страницу, все делал код, то есть это было автоматически, но пришлось пошаманить). Надо было брать другой сервис, потому что с кинопоиском просто нереально, не снижайте) Тестов у меня будет больше, но какая разница, лучше проверю в итоге.

In [745]:
def cleaner(review):
    b_result = ""
    for c in review:
        if c not in string.punctuation:
            b_result += c
    tokens = [w.lower() for w in word_tokenize(b_result) if w.isalpha()]
    return tokens

In [746]:
def bad_reviews(soup):    
    reviews = soup.find_all('div', {'class': 'response bad'})  
    review = reviews[0].find('span').text
    review = re.sub(r'\n', '', review)
    review = re.sub(r'\r', ' ', review)
    review = cleaner(review)
    negative.append(review)

In [747]:
def good_reviews(soup):    
    reviews = soup.find_all('div', {'class': 'response good'})
    review = reviews[0].find('span').text
    review = re.sub(r'\n|', '', review)
    review = re.sub(r'\r', ' ', review)
    review = cleaner(review)
    positive.append(review)

In [748]:
def neutral_reviews(soup):
    reviews = soup.find_all('div', {'class': 'response good'})
    review = reviews[1].find('span').text
    review = re.sub(r'\n|', '', review)
    review = re.sub(r'\r', ' ', review)
    review = cleaner(review)
    d = {}
    d['positive'] = review
    test.append(d)
    d = {}
    reviews = soup.find_all('div', {'class': 'response bad'})
    review = reviews[1].find('span').text
    review = re.sub(r'\n|', '', review)
    review = re.sub(r'\r', ' ', review)
    review = cleaner(review)
    d['negative'] = review
    test.append(d)

In [740]:
positive = []
negative = []
test = []

In [851]:
negative

'negative'

У всех фильмов структура ссылки на отзывы одинаковая, так что я упростил себе задачу.

In [ ]:
for id in films_ids:
    url = 'https://www.kinopoisk.ru'+id+'reviews/ord/date/status/all/perpage/200/'
    req = session.get(url, headers={'User-Agent': ua.random})
    page = req.text
    soup = BeautifulSoup(page, 'html.parser')
    good_reviews(soup)
    try:
        bad_reviews(soup)    
    except:
        pass
    try:
        neutral_reviews(soup)
    except:
        pass

Теперь начальные формы. Я не хочу делать в функции, так менее красиво имхо. Но могу/ А ещё можно было бы сделать список для каждого отзыва, но вроде это нам не нужно.

In [823]:
from pymorphy2 import MorphAnalyzer
morph = MorphAnalyzer()
negative_wordforms = []
for review in negative:
    n = 0
    for word in review:
        info = morph.parse(word)
        wordform = info[0].normal_form
        negative_wordforms.append(wordform)
    n+=1

In [824]:
morph = MorphAnalyzer()
positive_wordforms = []
for review in positive:
    n = 0
    for word in review:
        info = morph.parse(word)
        wordform = info[0].normal_form
        positive_wordforms.append(wordform)
    n+=1

In [825]:
morph = MorphAnalyzer()
test_wordforms = []
n = 0
for review in test:
    d = {}
    review_wordforms = []
    for list in review.values():
        for word in list:
            info = morph.parse(word)
            wordform = info[0].normal_form
            review_wordforms.append(wordform)
    for key, value in review.items():
        if key == 'negative':
            d['negative'] = review_wordforms
        else:
            d['positive'] = review_wordforms 
    test_wordforms.append(d)
    n+=1

Так. Я молодец. Можно переходить к третьему пункту. Сразу отправляю жабу.

In [ ]:
from IPython.display import Image
Image(filename =r'jaba.jpg')

Создание уникальных словарей для поз/нег уникальных слов. Возможно немного через жопу и с кучей словарей, но работает.

In [826]:
unique_positive = []
for word in positive_wordforms:
    if not word in negative_wordforms:
        unique_positive.append(word)

In [827]:
unique_positive_freqlist = {}
for word in unique_positive:
    try:
        unique_positive_freqlist[word] += 1
    except:
        unique_positive_freqlist[word] = 1

In [828]:
unique_positive_freqlist_cleared = {}
for key, value in unique_positive_freqlist.items():
    if value > 2:
        unique_positive_freqlist_cleared[key] = value

И для негативных отзывов

In [829]:
unique_negative = []
for word in negative_wordforms:
    if not word in positive_wordforms:
        unique_negative.append(word)

In [830]:
unique_negative_freqlist = {}
for word in unique_negative:
    try:
        unique_negative_freqlist[word] += 1
    except:
        unique_negative_freqlist[word] = 1

In [831]:
unique_negative_freqlist_cleared = {}
for key, value in unique_negative_freqlist.items():
    if value > 2:
        unique_negative_freqlist_cleared[key] = value

Функция определения тональности отзыва. Тут вроде все понятно

In [832]:
positive_counter = 0
negative_counter = 0
true = 0

for review in test_wordforms:
    positive_counter = 0
    negative_counter = 0
    for key, value in review.items():
        for word in value:
            if word in unique_positive_freqlist_cleared.keys():
                positive_counter += 1
            if word in unique_negative_freqlist_cleared.keys():
                negative_counter += 1
        
        if positive_counter > negative_counter:
            positive = 'positive'
            print('I think it is positive' + '\nTrue answer is: '+key)
            if key == positive:
                true+=1
        if positive_counter < negative_counter:
            negative = 'negative'
            print('I think it is negative' + '\nTrue answer is: '+key)
            if key == negative:
                true+=1
        if positive_counter == negative_counter:
            print("I don't know" + '\nTrue answer is: '+key)
        print('\n')

accuracy = (true / len(test_wordforms)) * 100
accuracy = str(accuracy) + '%'

I think it is negative
True answer is: positive


I think it is negative
True answer is: negative


I think it is negative
True answer is: positive


I think it is negative
True answer is: negative


I think it is negative
True answer is: positive


I think it is negative
True answer is: negative


I think it is positive
True answer is: positive


I think it is negative
True answer is: positive


I think it is negative
True answer is: negative


I think it is negative
True answer is: positive


I think it is negative
True answer is: negative


I think it is negative
True answer is: positive


I think it is negative
True answer is: negative


I think it is positive
True answer is: positive


I think it is negative
True answer is: negative


I think it is positive
True answer is: positive


I think it is positive
True answer is: negative


I think it is negative
True answer is: positive


I think it is negative
True answer is: negative


I think it is negative
True answer is: positive




In [833]:
print(accuracy)

58.620689655172406%


Что можно сделать? Повысить выборку, чтобы достичь большей точности. Сбалансировать размер словарей с уникальными словами для positive negative. Сделать значимость наиболее встречаемых слов выше.

# HW2 <br>
Попробуем биграммы

In [834]:
from pymystem3 import Mystem

In [727]:
def parcer(text):
    m = Mystem()
    lemmas = m.analyze(text)
    return(lemmas)

In [728]:
def tag_changer(tag):
    if tag == 'V':
        tag = re.sub('V', 'VERB', tag)

    elif tag == 'S':
        tag = re.sub('S', 'NOUN', tag)

    elif tag == 'PR':
        tag = re.sub('PR', 'PREP', tag)

    elif tag == 'A':
        tag = re.sub('A', 'ADJ', tag)

    elif tag == 'APRO':
        tag = re.sub('APRO', 'PRON', tag)

    elif tag == 'ADVPRO':
        tag = re.sub('ADVPRO', 'ADV', tag)

    elif tag == 'SPRO':
        tag = re.sub('SPRO', 'PRON', tag)

    return tag

In [865]:
def bigramm_maker(text):
    lemmas = parcer(text)
    bigramms = []
    temp = []
    for parsing in lemmas:
            if 'analysis' in parsing:
                woord = parsing['text']

                #это на случай иностранных слов

                try:
                    gr = parsing['analysis'][0]['gr']
                    pos = gr.split('=')[0].split(',')[0]
                    tag = tag_changer(pos)
                    temp.append({tag: woord})
                except:
                    pass

                try:
                    if list(temp[-2].keys())[0] == 'ADJ' and tag == 'NOUN':
                        bigramm = []
                        bigramm.append(list(temp[-2].values())[0])
                        bigramm.append(woord)
                        bigramms.append(bigramm)
                        temp = []
                except:
                    pass

                try:
                    if list(temp[-2].keys())[0] == 'PREP' and tag == 'NOUN':
                        bigramm = []
                        bigramm.append(list(temp[-2].values())[0])
                        bigramm.append(woord)
                        bigramms.append(bigramm)
                        temp = []    
                except:
                    pass

                try:
                    if list(temp[-2].keys())[0] == 'VERB' and tag == 'NOUN':
                        bigramm = []
                        bigramm.append(list(temp[-2].values())[0])
                        bigramm.append(woord)
                        bigramms.append(bigramm)
                        temp = []
                except:
                    pass
    return bigramms

Делаю строку для удобства анализа. Для тестовых отзывов получится список словарей, где ключ - это тональность, а значение - это список биграммы

In [842]:
posit = ''
for text in positive_wordforms:
    posit = posit + ' ' + text

In [845]:
negat = ''
for text in negative_wordforms:
    negat = negat + ' ' + text

In [911]:
test_done = []
for text in test_wordforms:
    test_ = ''
    if 'positive' in text:
        for word in text['positive']:
            test_ = test_ + ' ' + word
        tested_bigramms = bigramm_maker(test_)
        test_done.append({'positive': tested_bigramms})
    else:
        for word in text['negative']:
            test_ = test_ + ' ' + word
        tested_bigramms = bigramm_maker(test_)
        test_done.append({'negative': tested_bigramms})


In [866]:
pos_bigramms = bigramm_maker(posit)

In [867]:
neg_bigramms = bigramm_maker(negat)

Уникальные биграммы для поз

In [950]:
b_unique_positive = []
for bigramm in pos_bigramms:
    if not bigramm in neg_bigramms:
        b_unique_positive.append(bigramm)

In [951]:
b_unique_positive_freqlist = {}
for bigramm in b_unique_positive:
    bigramm = bigramm[0] + ' ' + bigramm[1]
    try:
        b_unique_positive_freqlist[bigramm] += 1
    except:
        b_unique_positive_freqlist[bigramm] = 1

In [952]:
b_unique_positive_freqlist_cleared = {}
for key, value in b_unique_positive_freqlist.items():
    if value > 1:
        b_unique_positive_freqlist_cleared[key] = value

Уникальные биграммы для нег

In [954]:
b_unique_negative = []
for bigramm in neg_bigramms:
    if not bigramm in pos_bigramms:
        b_unique_negative.append(bigramm)

In [955]:
b_unique_negative_freqlist = {}
for bigramm in b_unique_negative:
    bigramm = bigramm[0] + ' ' + bigramm[1]
    try:
        b_unique_negative_freqlist[bigramm] += 1
    except:
        b_unique_negative_freqlist[bigramm] = 1

In [956]:
b_unique_negative_freqlist_cleared = {}
for key, value in b_unique_negative_freqlist.items():
    if value > 1:
        b_unique_negative_freqlist_cleared[key] = value

Проверка точности

In [973]:
positive_counter = 0
negative_counter = 0
true = 0

for review in test_done:
    positive_counter = 0
    negative_counter = 0
    for key, value in review.items():
        for bigramm in value:
            word = bigramm[0] + ' ' + bigramm[1]
            if word in b_unique_positive_freqlist_cleared.keys():
                positive_counter += 1
            if word in b_unique_negative_freqlist_cleared.keys():
                negative_counter += 1
        
        if positive_counter > negative_counter:
            positive = 'positive'
            print('I think it is positive' + '\nTrue answer is: '+key)
            if key == positive:
                true+=1
        if positive_counter < negative_counter:
            negative = 'negative'
            print('I think it is negative' + '\nTrue answer is: '+key)
            if key == negative:
                true+=1
        if positive_counter == negative_counter:
            print("I don't know" + '\nTrue answer is: '+key)
        print('\n')

accuracy = (true / len(test_done)) * 100
accuracy = str(accuracy) + '%'

I think it is negative
True answer is: positive


I think it is negative
True answer is: negative


I don't know
True answer is: positive


I think it is negative
True answer is: negative


I don't know
True answer is: positive


I think it is positive
True answer is: negative


I don't know
True answer is: positive


I think it is positive
True answer is: positive


I think it is positive
True answer is: negative


I think it is negative
True answer is: positive


I think it is negative
True answer is: negative


I think it is negative
True answer is: positive


I don't know
True answer is: negative


I think it is negative
True answer is: positive


I don't know
True answer is: negative


I don't know
True answer is: positive


I don't know
True answer is: negative


I think it is negative
True answer is: positive


I think it is negative
True answer is: negative


I think it is negative
True answer is: positive


I think it is negative
True answer is: negative


I think it is negati

In [972]:
accuracy

'37.93103448275862%'

Такая себе точность, слишком маленькая выборка биграмм.